<a href="https://colab.research.google.com/github/Shruti02112/Mediscript/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/openai/whisper.git
!pip install transformers langdetect

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-sjku1198
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-sjku1198
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
!wget -q https://huggingface.co/kormilitzin/en_core_med7_trf/resolve/main/en_core_med7_trf-any-py3-none-any.whl -O en_core_med7_trf-0.0.1-py3-none-any.whl

!pip install --no-deps en_core_med7_trf-0.0.1-py3-none-any.whl
!pip install spacy==3.7 spacy-transformers==1.3.8

Processing ./en_core_med7_trf-0.0.1-py3-none-any.whl
en-core-med7-trf is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [3]:
!pip install spacy-transformers==1.1.6


  Using cached spacy_transformers-1.1.6-py2.py3-none-any.whl.metadata (6.0 kB)
  Using cached transformers-4.19.4-py3-none-any.whl.metadata (73 kB)
  Using cached tokenizers-0.12.1.tar.gz (220 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Using cached spacy_transformers-1.1.6-py2.py3-none-any.whl (51 kB)
Using cached transformers-4.19.4-py3-none-any.whl (4.2 MB)
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (tokenizers)


In [4]:
import whisper
import spacy
from transformers import pipeline
from langdetect import detect
from collections import defaultdict

In [5]:

#Load models
asr_model = whisper.load_model("base")
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-mul-en")

print("All models loaded successfully!")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cpu


All models loaded successfully!


In [6]:
import spacy
nlp = spacy.load("en_core_med7_trf")  # or use a direct path if needed
print("med7 loaded successfully")




/usr/local/lib/python3.12/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_med7_trf' (3.4.2.1) was trained with spaCy v3.4.2 and may not be 100% compatible with the current version (3.7.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/spacy_transformers/layers/hf_shim.py:137: UserWarning: Error loading saved torch state_dict with strict=True, likely due to differences between 'transformers' versions. Attempting to load with strict=False as a fallback...

If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current 'transformers' and 'spacy-transformers' versions. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


med7 loaded successfully


In [7]:
from google.colab import files

print("Upload a WAV or MP3 audio file...")
uploaded = files.upload()
filename = next(iter(uploaded))


Upload a WAV or MP3 audio file...


Saving Recording.m4a to Recording.m4a


In [10]:
def transcribe_audio(path):
    print("Transcribing with Whisper...")
    result = asr_model.transcribe(path)
    return result['text']

def translate_to_english(text):
    lang = detect(text)
    print(f"Detected language: {lang}")
    if lang == "en":
        return text
    print("Translating to English...")
    return translator(text)[0]['translation_text']

def extract_prescription_entities(text):
    doc = nlp(text)
    meds = []
    current = defaultdict(str)

    for ent in doc.ents:
        if ent.label_ == "DRUG" and current.get("DRUG"):
            meds.append(dict(current))
            current = defaultdict(str)
        current[ent.label_] += ent.text.strip() + " "

    if current:
        meds.append(dict(current))

    for med in meds:
        for key in med:
            med[key] = med[key].strip()

    return meds


In [11]:

# Run full pipeline
transcribed = transcribe_audio(filename)
print("\nTranscribed text:\n", transcribed)

translated = translate_to_english(transcribed)
print("\nTranslated text:\n", translated)

entities = extract_prescription_entities(translated)
print("\nExtracted Prescriptions:")

for i, med in enumerate(entities, 1):
    print(f"\nMedication #{i}")
    for k, v in med.items():
        print(f"  {k}: {v.strip()}")


Transcribing with Whisper...


/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



Transcribed text:
  The patient was prescribed paracetamol 500 mg or early twice daily for 5 days. I am oxalene 250 gm or 50 mg per 5 ml suspension, 10 ml or early 3 times a day for 7 days. Administrator, administer, metromorphine, 850 mg once daily with brake pass for 30 days.
Detected language: en

Translated text:
  The patient was prescribed paracetamol 500 mg or early twice daily for 5 days. I am oxalene 250 gm or 50 mg per 5 ml suspension, 10 ml or early 3 times a day for 7 days. Administrator, administer, metromorphine, 850 mg once daily with brake pass for 30 days.


/usr/local/lib/python3.12/dist-packages/thinc/shims/pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):



Extracted Prescriptions:

Medication #1
  DRUG: paracetamol
  STRENGTH: 500 mg
  FREQUENCY: twice daily
  DURATION: for 5 days

Medication #2
  DRUG: oxalene
  STRENGTH: 250 gm 50 mg
  FORM: suspension
  DOSAGE: 10 ml
  FREQUENCY: 3 times a day
  DURATION: for 7 days

Medication #3
  DRUG: metromorphine
  STRENGTH: 850 mg
  FREQUENCY: once daily with brake pass
  DURATION: for 30 days
